## This program is to build up the CCAE+RUL research step by step ##



In [ ]:
#  import tools 
from sklearn.model_selection import train_test_split
from keras.src.models import Model
from keras.src.saving.saving_api import load_model
from keras.src.layers import Input, Conv1D, Dense, concatenate, RepeatVector, MaxPooling1D, Activation ,UpSampling1D, Conv1DTranspose
from keras.src.utils import plot_model
import numpy as np 
import pandas as pd
from openpyxl import Workbook
import matplotlib.pyplot as plt
import os
import sys
import csv
import time
from IPython import embed

from rul_features.rul_data_read import read_rul_data

from test_algs.CCAE_ntu_rul import get_initial_files_datalist, CCAE_train
from test_algs.CCAE_ntu_rul import CCAE_model_application, CCAE_model_build_train

* First step: check the file status *

In [ ]:
# 開啟互動模式
%matplotlib widget

train_file_list=[]

# Data set folder 
NTU_dataset_path = r'D:\OneDrive\RUL HI Reasearch Result\Data_sets\NTU_RUL_v2_data\Acc_life_test_data\Organized_Data'

# 測試階段只引入 1800rpm的資料

# -----------------------10psi data--------------------

collect_setting = r"Load_Free\Speed_1800\Pressure_10psi"  
train_file_list.append(os.path.join(NTU_dataset_path, collect_setting, r"0312_V1800_10P_FREE0"))
train_file_list.append(os.path.join(NTU_dataset_path, collect_setting, r"0313_V1800_10P_FREE1"))
train_file_list.append(os.path.join(NTU_dataset_path, collect_setting, r"0314_V1800_10P_FREE2"))

collect_setting = r"Load_1V\Speed_1800\Pressure_10psi"
train_file_list.append(os.path.join(NTU_dataset_path, collect_setting, r"10kg_1V_1800rpm_1"))
train_file_list.append(os.path.join(NTU_dataset_path, collect_setting, r"15kg_1V_1800rpm_1"))
train_file_list.append(os.path.join(NTU_dataset_path, collect_setting, r"15kg_1V_1800rpm_2"))

collect_setting = r"Load_2V\Speed_1800\Pressure_10psi"
train_file_list.append(os.path.join(NTU_dataset_path, collect_setting, r"0509_V1800_10P_2_1"))

# -----------------------5psi data--------------------

collect_setting = r"Load_1V\Speed_1800\Pressure_5psi"
# train_file_list.appeng(os.path.join(NTU_dataset_path, collect_setting, r"5psi_1V_1_3min"))
train_file_list.append(os.path.join(NTU_dataset_path, collect_setting, r"06kg_1V_1800rpm_2"))

collect_setting= r"Load_0V\Speed_1800\Pressure_5psi"
train_file_list.append(os.path.join(NTU_dataset_path, collect_setting, r"0313_V1800_5P_FREE2"))

collect_setting= r"Load_2V\Speed_1800\Pressure_5psi"
train_file_list.append(os.path.join(NTU_dataset_path, collect_setting, r"0402_V1800_5P_2_1"))

# 確認 train_file_list 中的所有路徑是否存在
for path in train_file_list:
    if not os.path.exists(path):
        print(f"檔案不存在: {path}")
    else:
        print(f"檔案存在: {path}")





In [ ]:
my_ccae_model1,_,__cached__ = CCAE_model_build_train.build_CCAE_model([1], sequence_length=1024, condition_dim=1,
                                                        model_name='V2I2_in_V2I2_out.keras', file_name='V2I2_in_V2I2_out_model.png')
# 顯示模型結構圖
plot_model(my_ccae_model1, to_file='V2I2_in_V2I2_out_model.png', show_shapes=True, show_layer_names=True)


In [ ]:


my_ccae_model2,_ ,_ = CCAE_model_build_train.build_CCAE_model([1,1], sequence_length=1024, condition_dim=1,
                                                        model_name='I2_in_I2_out.keras', file_name='I2_in_I2_out_model.png')
# 顯示模型結構圖
plot_model(my_ccae_model2, to_file='I2_in_I2_out.png', show_shapes=True, show_layer_names=True)

In [ ]:

my_ccae_model3 = CCAE_model_build_train.build_CCAE_model([1,1], sequence_length=1024, condition_dim=1,
                                                        model_name='V2_in_I2_out.keras', file_name='V2_in_I2_out_model.png')
# my_ccae_model = CCAE_model_build_train.build_CCAE_model([1,1,1,1], sequence_length=1024, condition_dim=1,
#                                                         model_name='I1_in_I1_out.keras', file_name='I1_in_I1_out_model.png')

** Step 2: train the ccae model **

In [ ]:
# CCAE_model_build_train.build_CCAE_model([1,1,1,1], sequence_length=1024, condition_dim=1)
from keras.src.utils import plot_model
from IPython.display import Image
from keras.src.layers import Concatenate
from IPython.display import HTML

input_dim_list=[1,1,1,1]
sequence_length=1024
condition_dim=1

inputs = []
condition_input = Input(shape=(condition_dim,), name='condition') 
repeated_condition = RepeatVector(sequence_length, name='repeated_condition')(condition_input)

# 合併所有time-series inputs
for idx, input_dim in enumerate(input_dim_list):
    input_layer = Input(shape=(sequence_length, input_dim), name=f'series_{idx}')
    inputs.append(input_layer)

# 將所有輸入通道與 repeated_condition 在最後一個維度上合併
encoder_input = Concatenate(axis=-1, name='full_encoder_input')(inputs + [repeated_condition])

# Encoder
x = Conv1D(filters=64, kernel_size=64, strides=16, padding='same')(encoder_input)
x = MaxPooling1D(pool_size=2)(x)
x = Activation('relu')(x)

x = Conv1D(filters=32, kernel_size=3, padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Activation('relu')(x)

x = Conv1D(filters=16, kernel_size=3, padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
encoded = Activation('relu')(x)

encoder_model = Model(inputs=inputs+[condition_input], outputs=encoded, name='Encoder')
# encoder_model.summary()
# plot_model(encoder_model, show_shapes=True, show_layer_names=True)



# Decoder
decoder_input = Input(shape=(encoder_model.output_shape[1], encoder_model.output_shape[2]), name='encoder_output')
decoder_condition_input = Input(shape=(condition_dim,), name='decoder_condition')
repeated_decoder_condition = RepeatVector(encoded.shape[1])(decoder_condition_input)
decoder_merge_input = concatenate([decoder_input, repeated_decoder_condition], name='merged_decoder_input')

# 重建tensor 
x = Conv1DTranspose(filters=16, kernel_size=3, padding='same')(decoder_merge_input)
x = UpSampling1D(2)(x)
x = Activation('relu')(x)

x = Conv1DTranspose(filters=32, kernel_size=3, padding='same')(x)
x = UpSampling1D(2)(x)
x = Activation('relu')(x)

x = Conv1DTranspose(filters=64, kernel_size=64, strides=16, padding='same')(x)
x = UpSampling1D(2)(x)
x = Activation('tanh')(x)

output_list = [Dense(dim, activation='linear', name=f'output_{idx}')(x) for idx, dim in enumerate(input_dim_list)]
decoder_model = Model(inputs=[decoder_input, decoder_condition_input], outputs=output_list, name='Decoder')
# decoder_model.summary()
# plot_model(decoder_model, show_shapes=True, show_layer_names=True)

# 編碼輸入：所有 series input + condition input
full_encoder_inputs = inputs + [condition_input]
encoded = encoder_model(full_encoder_inputs)

# 將 encoder 輸出與 condition 傳入 decoder
decoder_outputs = decoder_model([encoded, condition_input])

# 建立完整模型
full_model = Model(inputs=full_encoder_inputs, outputs=decoder_outputs, name=f'CCAE_model {encoder_input.shape[2]-1} in {len(output_list)} out')

# 顯示模型結構圖
plot_model(full_model, show_shapes=True, show_layer_names=True)




In [ ]:
len(output_list)

In [ ]:

# Full CCAE model

model = Model(inputs=inputs+[condition_input], outputs=output_list)
model.compile(optimizer='adam', loss='mse')
# model.summary()

plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
len(output_list)


In [ ]:


# use the first 10% data for CCAE training or filtering 
Pre_train_set=0.1
# Create training data set 
motor_data_list=get_initial_files_datalist(train_file_list, Pre_train_set,
                                            input_name="Current alpha downsample")


# Shuffle the training data randomly
np.random.seed(42)  # Set random seed for reproducibility 
np.random.shuffle(motor_data_list)

# 訓練CCAE 模型


# CCAE_train(Motor_data = motor_data_list,
#                         model_name = "0524_ItoI_model_downsample.keras")

CCAE_train(Motor_data = motor_data_list,
                        model_name = "0608_ItoI_all_1800.keras")


** Step 3: Apply the ccae model **

In [ ]:
# 指定模型名稱
# model_name = "0523_test_model.keras"
# model_name = "0524_ItoI_model_downsample.keras"
model_name= "0608_ItoI_all_1800.keras"


ccae_model=load_model(model_name)


# 儲存所有 ccae 物件
my_ccae_objects=[]
for i in range(len(train_file_list)):
    my_ccae=CCAE_model_application(model=ccae_model, file_path=train_file_list[i])
    mse_errors = my_ccae.predict_mse_byfile()
    my_ccae.get_basic_timelist()
    my_ccae_objects.append(my_ccae)


In [ ]:
# 根據 my_ccae_objects 的數量動態產生 (n, 1) 的 subplot
n = len(my_ccae_objects)
fig, axes = plt.subplots(n, 1, figsize=(12, 3 * n), sharex=True)

if n == 1:
    axes = [axes]  # 保證 axes 可迭代

for idx, (my_ccae, ax) in enumerate(zip(my_ccae_objects, axes)):
    ax.plot(my_ccae.basic_timelist["Time stamps"], my_ccae.ccae_repport["mse_avg_byfile"],
            label=f"File: {my_ccae.file_path.split(os.sep)[-1]}")
    ax.set_ylabel("MSE Error")
    ax.legend()
    ax.grid(True)
    ax.set_ylim(0, 0.005)

axes[-1].set_xlabel("Sample Index")
plt.tight_layout()
plt.show()

In [ ]:

# 繪製並重疊每次實驗的 MSE Errors
plt.figure(figsize=(12, 8))
plt.xlabel("Sample Index")
plt.ylabel("MSE Error")
for my_ccae in (my_ccae_objects):
    plt.plot(my_ccae.basic_timelist["Time stamps"],my_ccae.ccae_repport["mse_avg_byfile"],
             label=f"File: {my_ccae.file_path.split(os.sep)[-1]}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show(block=True)
plt.savefig("mse_errors_plot.png")
plt.ylim(0, 0.005)

In [ ]:
for my_ccae in (my_ccae_objects):
    plt.figure(figsize=(12, 8))
    plt.xlabel("Sample Index")
    plt.ylabel("MSE Error")
    plt.plot(my_ccae.basic_timelist["Time stamps"],my_ccae.ccae_repport["mse_avg_byfile"],
            label=f"File: {my_ccae.file_path.split(os.sep)[-1]}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show(block=True)